In [2]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import quandl as qd
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from pandas_datareader import data
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import os
import glob
from time import sleep

In [3]:
# Importing cross currency sheet
cryptos = pd.read_csv('crosspairs.csv')

'''
for idx, row in cryptos.iterrows():
    symbol = row['Symbol']
    exchanges = row.dropna().index.values[2:]
    
exchanges
'''

"\nfor idx, row in cryptos.iterrows():\n    symbol = row['Symbol']\n    exchanges = row.dropna().index.values[2:]\n    \nexchanges\n"

In [4]:
# Pull from Cryptopia
def pulltopia(symbol,base):
    # Fields:
    # AskPrice BaseVolume BidPrice BuyBaseVolume BuyVolume Change
    # Close High Label LastPrice Low Open SellBaseVolume 
    # SellVolume TradePairId Volume
    
    topiasite = 'https://www.cryptopia.co.nz/api/GetMarket/' + symbol + '_' + base + '/0.001'
    response = requests.get(topiasite)
    json_data = json.loads(response.text)
    return pd.DataFrame(json_data['Data'], index = [0])

# Pull from kucoin
def pullkucoin(symbol,base):
    kubuy = 'https://api.kucoin.com/v1/' + symbol + '-' + base + '/open/orders-buy/'
    kusell = 'https://api.kucoin.com/v1/' + symbol + '-' + base + '/open/orders-sell/'
    respone = requests.get(kubuy)
    json_data_buy = json.loads(respone.text)
    response = requests.get(kusell)
    json_data_sell = json.loads(response.text)
    return pd.DataFrame({'BidPrice':[x[0] for x in json_data_buy['data']],
                         'BuyBaseVolume': [x[1] for x in json_data_buy['data']],
                         'AskPrice': [x[0] for x in json_data_sell['data']],
                         'SellBaseVolume': [x[1] for x in json_data_sell['data']]},
                        index = range(len(json_data_buy['data'])))

# Pull from Bittrex
def trexpull(symbol,base):
    trexsite = 'https://bittrex.com/api/v1.1/public/getorderbook?market=' + base + '-' + symbol + '&type=both'
    response = requests.get(trexsite)
    json_data = json.loads(response.text)
    return pd.concat([pd.DataFrame({'BidPrice': [x['Rate'] for x in json_data['result']['buy']],
                                    'BuyBaseVolume': [x['Quantity'] for x in json_data['result']['buy']]},
                                   index = range(len(json_data['result']['buy']))),
                      pd.DataFrame({'AskPrice': [x['Rate'] for x in json_data['result']['sell']],
                                    'SellBaseVolume': [x['Quantity'] for x in json_data['result']['sell']]},
                                   index = range(len(json_data['result']['sell'])))],
                     axis = 1)

# Pull from Quoine
def quoinepull(id):
    # currencypair id
    # UBTCETH-->75
    # UBTCBTC-->74
    # BTCUSD-->1
    # QASHBTC-->52
    # ETHBTC-->37
    # ETHUSD-->27
    # NEOUSD-->53
    # 
    quoinesite = 'https://api.quoine.com/products/' + str(id) + '/price_levels'
    response = requests.get(quoinesite)
    json_data = json.loads(response.text)
    return pd.DataFrame({'BidPrice': [x[0] for x in json_data['buy_price_levels']],
                         'BuyBaseVolume': [x[1] for x in json_data['buy_price_levels']],
                         'AskPrice': [x[0] for x in json_data['sell_price_levels']],
                         'SellBaseVolume': [x[1] for x in json_data['sell_price_levels']]},
                        index = range(len(json_data['buy_price_levels'])))

In [29]:
# Arbitrage from Kucoin and Trex

def kutrexarb(symbol,base):
    ku = pullkucoin(symbol,base)
    ku.columns = ['ku_' + colname for colname in ku]
    trex = trexpull(symbol,base)[:6]
    trex.columns = ['trex_' + colname for colname in trex]
    return pd.concat([ku,trex],axis = 1)

def checkarbs():
    pairs = pd.read_csv('kutrex.csv')
    currencypairs = []
    buyex = []
    sellex = []
    askprice = []
    bidprice = []
    arbamount = []
    for idx, row in pairs.iterrows():
        symbol = row['Symbol']
        base = row['Base']
        tempseries = kutrexarb(symbol,base).iloc[0]
        
        if tempseries['ku_AskPrice'] < tempseries['trex_BidPrice']:
            # Buy on Ku and sell on Trex
            currencypairs = currencypairs + [symbol + base]
            buyex = buyex + ['Ku']
            sellex = sellex + ['Trex']
            askprice = askprice + [tempseries['ku_AskPrice']]
            bidprice = bidprice + [tempseries['trex_BidPrice']]
            arbamount = arbamount + [tempseries['trex_BidPrice']/tempseries['ku_AskPrice']-1]
        elif tempseries['trex_AskPrice'] < tempseries['ku_BidPrice']:
            # Buy on Trex ad sell on Ku
            currencypairs = currencypairs + [symbol + base]
            buyex = buyex + ['Trex']
            sellex = sellex + ['Ku']
            askprice = askprice + [tempseries['trex_AskPrice']]
            bidprice = bidprice + [tempseries['ku_BidPrice']]
            arbamount = arbamount + [tempseries['ku_BidPrice']/tempseries['trex_AskPrice']-1]
        else:
            continue
        sleep(0.1)
    
    df = pd.DataFrame({'BuyExchange': buyex,
                      'SellExchange': sellex,
                      'AskPrice': askprice,
                      'BidPrice': bidprice,
                      'ArbReturn': arbamount},
                     index = currencypairs)
    return df.sort_values(['ArbReturn'])

def ethbtcconvert():
    kutrex = kutrexarb('ETH','BTC')[['ku_AskPrice','ku_BidPrice',
                                    'trex_AskPrice','trex_BidPrice']]
    topia = pulltopia('ETH','BTC')[['AskPrice','BidPrice']]
    topia.columns = ['topia_AskPrice','topia_BidPrice']
    quoine = quoinepull(37)[['AskPrice','BidPrice']]
    quoine.columns = ['quoine_AskPrice','quoine_BidPrice']
    
    askdf = pd.concat([kutrex[['ku_AskPrice','trex_AskPrice']],
                      topia[['topia_AskPrice']],
                      quoine[['quoine_AskPrice']]],
                     axis = 1)
    
    bidf = pd.concat([kutrex[['ku_BidPrice','trex_BidPrice']],
                      topia[['topia_BidPrice']],
                      quoine[['quoine_BidPrice']]],
                     axis = 1)
    
    return askdf.iloc[0], bidf.iloc[0]

In [85]:
ask, bid = ethbtcconvert()

print(ask)
print(bid)

ku_AskPrice          0.098397
trex_AskPrice        0.098504
topia_AskPrice         0.0982
quoine_AskPrice    0.09396000
Name: 0, dtype: object
ku_BidPrice           0.09781
trex_BidPrice       0.0980186
topia_BidPrice      0.0979115
quoine_BidPrice    0.09130000
Name: 0, dtype: object


In [79]:
quoinepull(37)

,AskPrice,BidPrice,BuyBaseVolume,SellBaseVolume
0,0.09400000,0.09180000,0.90383728,0.74382979
1,0.09489999,0.09160000,1.00000000,2.00000000
2,0.09490000,0.09140010,97.91647731,5.00000000
3,0.09500000,0.09140000,23.24012291,2.10000000
4,0.09599000,0.09102089,0.23939087,50.00000000
5,0.09600000,0.09013000,0.49000000,0.10000000
6,0.09700000,0.08970000,1.11500000,0.10000000
7,0.09714000,0.08952000,0.14000000,25.00000000
8,0.09800000,0.08951100,0.10000000,0.10000000
9,0.09899000,0.08951000,5.46794659,25.00000000


In [72]:
checkarbs()

,ArbReturn,AskPrice,BidPrice,BuyExchange,SellExchange
SNTETH,0.000343,0.000321,0.000321,Trex,Ku
LTCBTC,0.000919,0.017784,0.017800,Ku,Trex
DASHBTC,0.001207,0.074699,0.074789,Trex,Ku
DGBBTC,0.001471,0.000007,0.000007,Trex,Ku
POWRETH,0.001881,0.001021,0.001023,Ku,Trex
DGBETH,0.002865,0.000070,0.000070,Trex,Ku
ETHBTC,0.003950,0.098799,0.099189,Trex,Ku
PAYETH,0.004688,0.002939,0.002953,Trex,Ku
OMGBTC,0.005318,0.001730,0.001739,Ku,Trex
CVCBTC,0.005348,0.000075,0.000075,Ku,Trex
